In [ ]:
!pip -q install -U "transformers>=4.41.0" "accelerate>=0.30.0" "bitsandbytes>=0.43.0" "peft>=0.11.1" datasets evaluate sentencepiece wandb

import torch, subprocess, sys, platform
print("PyTorch:", torch.__version__)
!nvidia-smi || echo "Pas de GPU NVIDIA (ok en Colab GPU)."


In [ ]:
from huggingface_hub import notebook_login
notebook_login()  


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"  
PROJECT_DIR = "lora-mistral-humor"
DATA_PATH   = "persona_qa (3).jsonl"  
HF_REPO_ID  = "ton-username/mistral-7b-humorous-lora"

SEED = 42
MAX_LEN = 512
NUM_EPOCHS = 3
LR = 2e-4
TRAIN_BS = 4
EVAL_BS = 4
GRAD_ACCUM = 1
WARMUP_RATIO = 0.03
LOG_STEPS = 50

import random, numpy as np
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)


In [ ]:
from datasets import load_dataset
data = load_dataset("json", data_files=DATA_PATH)  
ds = data["train"].train_test_split(test_size=0.1, seed=SEED)
train_raw, eval_raw = ds["train"], ds["test"]
len(train_raw), len(eval_raw), train_raw[0]


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
print("BOS/EOS:", tokenizer.bos_token, tokenizer.eos_token)
print("Has chat_template?", tokenizer.chat_template is not None)


In [ ]:
MAX_LEN = 512  

def build_prompt(instruction: str):
    return tokenizer.apply_chat_template(
        [{"role": "user", "content": instruction.strip()}],
        tokenize=False, add_generation_prompt=True
    )

def encode_example(example):
    prompt = build_prompt(example["instruction"])
    answer = (example.get("output") or "").strip()

    full = tokenizer(
        prompt + answer,
        max_length=MAX_LEN,
        truncation=True,
        padding="max_length",      
        add_special_tokens=False,
    )
    input_ids = full["input_ids"]
    attention_mask = full["attention_mask"]
    labels = input_ids.copy()

    for i, m in enumerate(attention_mask):
        if m == 0:
            labels[i] = -100

    prompt_ids = tokenizer(prompt, add_special_tokens=False)["input_ids"]
    pl = min(len(prompt_ids), len(labels))  
    for i in range(pl):
        labels[i] = -100

    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}


In [ ]:
cols_to_remove = train_raw.column_names
train_ds = train_raw.map(encode_example, remove_columns=cols_to_remove, desc="Tokenizing train (fixed)")
eval_ds  = eval_raw.map(encode_example,  remove_columns=cols_to_remove, desc="Tokenizing eval (fixed)")


In [ ]:
def check_lengths(ds, name):
    bad = []
    for i, ex in enumerate(ds):
        if len(ex["input_ids"]) != MAX_LEN or len(ex["labels"]) != MAX_LEN or len(ex["attention_mask"]) != MAX_LEN:
            bad.append(i)
    print(f"{name}: OK={len(ds)-len(bad)} / {len(ds)} ; BAD={len(bad)}")
    return bad

bad_train = check_lengths(train_ds, "train")
bad_eval  = check_lengths(eval_ds,  "eval")


In [ ]:
from transformers import default_data_collator, TrainingArguments, Trainer

data_collator = default_data_collator  

args = TrainingArguments(
    output_dir=PROJECT_DIR,
    overwrite_output_dir=True,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=TRAIN_BS,
    per_device_eval_batch_size=EVAL_BS,
    gradient_accumulation_steps=GRAD_ACCUM,
    learning_rate=LR,
    warmup_ratio=WARMUP_RATIO,
    logging_steps=LOG_STEPS,
    logging_strategy="steps",
    eval_strategy="epoch",          
    save_strategy="no",
    fp16=True, bf16=False,
    report_to="none",
    seed=SEED,
    remove_unused_columns=False,    
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=data_collator,
)


In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16, 
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    quantization_config=bnb_cfg,
    trust_remote_code=True,
)
model.config.use_cache = False  
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()  

from peft import prepare_model_for_kbit_training
model = prepare_model_for_kbit_training(model)  

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_cfg = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
)
model = get_peft_model(model, lora_cfg)

def print_trainables(m):
    trainable, total = 0, 0
    for p in m.parameters():
        n = p.numel()
        total += n
        if p.requires_grad:
            trainable += n
    print(f"Trainable params: {trainable:,} / {total:,} ({trainable/total*100:.2f}%)")
print_trainables(model)


In [ ]:
trainer.train()


In [ ]:
import math, re
eval_out = trainer.evaluate()
eval_loss = eval_out["eval_loss"]
ppl = math.exp(eval_loss) if eval_loss < 50 else float("inf")
print(f"Eval loss: {eval_loss:.4f} — Perplexity: {ppl:.2f}")

def normalize_text(s):
    s = s.strip().lower()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[^\w\s]", "", s)
    return s

from tqdm.auto import tqdm
model.eval(); model.config.use_cache = True

preds, refs = [], []
for ex in tqdm(eval_raw):
    prompt = tokenizer.apply_chat_template(
        [{"role":"user","content": ex["instruction"]}],
        tokenize=False, add_generation_prompt=True
    )
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=128, do_sample=False, temperature=0.0)
    gen = tokenizer.decode(out[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip()
    preds.append(gen)
    refs.append(ex["output"].strip())

em = np.mean([normalize_text(p)==normalize_text(r) for p,r in zip(preds,refs)])
print(f"Exact Match (normalisé) sur test: {em*100:.2f}%")

for i in range(3):
    print("\nQ:", eval_raw[i]["instruction"])
    print("REF:", refs[i])
    print("GEN:", preds[i])


In [ ]:
SAVE_DIR = "persona_humor_lora"
model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)
!ls -lh {SAVE_DIR}


In [ ]:
HF_REPO_ID = "tonioexe/mistral-7b-humorous-lora" 

model.push_to_hub(HF_REPO_ID)
tokenizer.push_to_hub(HF_REPO_ID)
print("Poussé sur:", HF_REPO_ID)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, device_map="auto",
    quantization_config=bnb_cfg, trust_remote_code=True
)
tok  = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

model_peft = PeftModel.from_pretrained(base, HF_REPO_ID)
model_peft.eval(); model_peft.config.use_cache = True

def chat(question):
    prompt = tok.apply_chat_template([{"role":"user","content": question}], tokenize=False, add_generation_prompt=True)
    inputs = tok([prompt], return_tensors="pt").to(model_peft.device)
    out = model_peft.generate(**inputs, max_new_tokens=128, do_sample=True, top_p=0.9, temperature=0.7)
    return tok.decode(out[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip()

print(chat("Quick snack ideas for gut health?"))


In [ ]:
merged = model_peft.merge_and_unload()  
CPU_DIR = "mistral7b_humor_merged_fp16"
merged.save_pretrained(CPU_DIR)
tok.save_pretrained(CPU_DIR)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    device_map="auto",
    load_in_4bit=True,
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

model = PeftModel.from_pretrained(base_model, "tonioexe/mistral-7b-humorous-lora")
model.eval()

def chat(question):
    prompt = f"<s>[INST] {question.strip()} [/INST]"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        top_p=0.9,
        temperature=0.8
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print(chat("Give me quick tips to survive Monday mornings?"))
print(chat("How do I politely cancel plans I never wanted in the first place?"))


In [ ]:
print(chat("How do I politely cancel plans I never wanted in the first place?"))
